In [1]:
import requests
import pandas as pd
import itertools
import time

In [2]:
def get_dist_table(df_coordinates, id="Falla"):
    """
    Se necesita un df de stops con columnas "id", "lat", "lon"
    """

    df_dist_query_tbl = df_coordinates.copy()
    # Elimiinar duplicados

    # Unir las coordenadas en listas
    rutas = [f"{row['lon']}_{row['lat']}" for index, row in df_dist_query_tbl.iterrows()]
    rutas = ", ".join(rutas)
    rutas = rutas.replace(", ", ";").replace("_", ",")

    # Hacer la petición
    while True:
        try:
            url = "http://router.project-osrm.org/table/v1/walking/"
            loc = rutas
            url_fin = '?annotations=distance'

            r = requests.get(url + loc + url_fin)
            res = r.json()

            # Crear un dataframe df_dist con todos los pares posibles     
            df_dist = pd.DataFrame(itertools.product(df_dist_query_tbl[id], repeat= 2), columns = ["ori", "des"])

            # Recorrer el df_dist e ir llenándolo con el res
            for io, o in enumerate(df_dist.des.unique()):
                df_dist.loc[df_dist.ori == o, "osrm_dist"] = res["distances"][io]
                # df_dist.loc[df_dist.ori == o, "time_"] = res["durations"][io]

            df_dist = df_dist[df_dist.ori!=df_dist.des]
            return df_dist, res
        
        except (requests.exceptions.RequestException, ValueError) as e:
            print(f"Error: {e}. \n Retrying in 5 seconds...")
            time.sleep(5)

In [3]:
df = pd.read_csv("fallas.csv")
df_dist, res = get_dist_table(df, id="Falla")

In [4]:
df_matrix = df_dist.pivot(index="ori", columns="des", values="osrm_dist").fillna(0)
df_matrix

des,Estació del Nord,Falla Almirante Cadarso-Conde de Altea,Falla Arzobispo Olaechea-San Marcelino,Falla Conde de Salvatierra-Cirilo Amorós-Mercat de Colón,Falla Convento Jerusalén-Matemático Marzal,Falla Cuba-Literato Azorín,Falla Císcar-Burriana,Falla Duc de Gaeta-Pobla de Farnals,Falla Espartero-Ramón y Cajal,Falla Exposición-Misser Mascó,...,Falla Plaza del Mercado Central,Falla Plaza del Pilar,Falla Quart-Extramurs-Velázquez,Falla Quart-Palomar,Falla Regne de Valencia-Duque de Calabria,Falla Reina-Paz-San Vicente (Falla Tio Pep),Falla Ribera-Convento Santa Clara,Falla San Vicente-Periodista Azzati,Falla Santa Genoveva Torres-Arquitecte Tolsà (la Nova de Orriols),Falla Sueca-Literato Azorín
ori,,,,,,,,,,,,,,,,,,,,,
Estació del Nord,0.0,2221.8,4124.1,1390.6,574.9,3100.0,2457.8,4504.7,1249.7,3922.9,...,1439.6,836.4,3081.4,1518.8,2272.8,942.0,359.1,642.1,4866.0,3176.9
Falla Almirante Cadarso-Conde de Altea,1085.7,0.0,5863.4,413.3,1632.2,1932.6,422.7,2507.1,2335.4,1925.3,...,2525.3,1922.1,4167.0,2604.4,667.5,2027.6,1444.8,1727.7,4597.0,2009.5
Falla Arzobispo Olaechea-San Marcelino,4034.5,4625.3,0.0,4814.5,3560.3,3633.2,4861.4,6457.8,3786.9,7445.6,...,4730.8,4127.6,4411.4,4810.0,4676.4,4392.3,3809.4,4092.4,8815.9,3710.1
Falla Conde de Salvatierra-Cirilo Amorós-Mercat de Colón,754.0,803.7,4878.1,0.0,1328.9,1918.8,1223.9,3086.6,2003.7,1997.8,...,2193.6,1590.4,3835.4,2272.8,1325.6,1696.0,1113.1,1396.1,4669.5,1995.6
Falla Convento Jerusalén-Matemático Marzal,895.0,2276.4,4178.7,2465.5,0.0,3154.6,2512.4,4559.3,1435.3,3977.5,...,1723.4,1120.1,3346.1,1802.5,2327.5,1252.7,669.9,952.8,5149.7,3231.5
Falla Cuba-Literato Azorín,1328.2,1237.5,4573.9,1645.1,2459.0,0.0,1135.5,3686.7,2685.6,3703.3,...,2767.8,2164.6,4491.9,2847.0,950.5,2270.2,1687.3,1970.3,6395.1,720.0
Falla Císcar-Burriana,1159.6,623.8,5937.4,487.3,1706.2,2006.6,0.0,2581.1,2409.3,1999.3,...,2599.3,1996.0,4241.0,2678.4,741.5,2101.6,1518.7,1801.7,4671.0,2083.5
Falla Duc de Gaeta-Pobla de Farnals,3495.6,2689.3,6313.8,3007.9,4034.3,3316.7,2352.0,0.0,5359.2,1819.8,...,4935.2,4332.0,7165.5,5014.4,2167.0,4437.6,3854.7,4137.7,5356.0,3393.6
Falla Espartero-Ramón y Cajal,1225.8,1965.7,3868.1,2154.8,751.6,2843.9,2201.8,4248.7,0.0,3666.8,...,1079.1,475.9,2589.7,1158.3,2016.8,1583.6,1000.7,1283.7,4505.5,2920.8


In [5]:
# Save the matrix
df_matrix.to_csv("df_matrix.csv", index=True)

In [ ]:
import pandas as pd
import requests
import itertools
import time

def get_dist_table(df_coordinates, id="Falla", route_profile="foot"):
    """
    Se necesita un df de stops con columnas "id", "lat", "lon"
    Obtiene la matriz de distancias y las rutas entre puntos usando OSRM.
    """

    df_dist_query_tbl = df_coordinates.copy()
    
    # Eliminar duplicados
    df_dist_query_tbl = df_dist_query_tbl.drop_duplicates(subset=["lat", "lon"])

    # Crear todas las combinaciones de pares origen-destino
    df_dist = pd.DataFrame(itertools.product(df_dist_query_tbl[id], repeat=2), columns=["ori", "des"])

    # Agregar columnas vacías para distancia y geometría
    df_dist["osrm_dist"] = None
    df_dist["route_geometry"] = None  

    table_url = f"http://router.project-osrm.org/table/v1/{route_profile}/"
    route_url = f"http://router.project-osrm.org/route/v1/{route_profile}/"

    while True:
        try:
            #  - Obtener la matriz de distancias
            rutas = [f"{row['lon']},{row['lat']}" for _, row in df_dist_query_tbl.iterrows()]
            rutas = ";".join(rutas)  # Formato para OSRM
            response_table = requests.get(f"{table_url}{rutas}?annotations=distance")
            res_table = response_table.json()

            for io, o in enumerate(df_dist.des.unique()):
                df_dist.loc[df_dist.ori == o, "osrm_dist"] = res_table["distances"][io]

            df_dist = df_dist[df_dist.ori != df_dist.des]  # Eliminar distancias a sí mismo

            # 2️- Obtener rutas individuales para cada par origen-destino
            for index, row in df_dist.iterrows():

                ori_data = df_dist_query_tbl[df_dist_query_tbl[id] == row["ori"]].iloc[0]
                des_data = df_dist_query_tbl[df_dist_query_tbl[id] == row["des"]].iloc[0]

                loc = f"{ori_data['lon']},{ori_data['lat']};{des_data['lon']},{des_data['lat']}"
                response_route = requests.get(f"{route_url}{loc}?alternatives=true&steps=true&geometries=geojson&overview=full")
                res_route = response_route.json()

                # Agregar la geometría de la ruta al DataFrame
                if "routes" in res_route and len(res_route["routes"]) > 0:
                    # print(index)
                    df_dist.at[index, "geometry"] = res_route["routes"][0]["geometry"]["coordinates"]

            return df_dist

        except (requests.exceptions.RequestException, ValueError) as e:
            print(f"Error: {e}. \n Retrying in 5 seconds...")
            time.sleep(5)
        
df_routes = get_dist_table(df, id="Falla", route_profile="walking")
df_routes

,ori,des,osrm_dist,route_geometry
1,Falla Convento Jerusalén-Matemático Marzal,Falla Cuba-Literato Azorín,3154.6,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
2,Falla Convento Jerusalén-Matemático Marzal,Falla Sueca-Literato Azorín,3231.5,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
3,Falla Convento Jerusalén-Matemático Marzal,Falla Almirante Cadarso-Conde de Altea,2276.4,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
4,Falla Convento Jerusalén-Matemático Marzal,Falla Na Jordana,2410,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
5,Falla Convento Jerusalén-Matemático Marzal,Falla Plaza del Pilar,1120.1,"[[-0.379878, 39.466528], [-0.379727, 39.466493..."
...,...,...,...,...
955,Estació del Nord,Falla Justo Vilar-Mercado del Cabañal,5574.7,"[[-0.377226, 39.467629], [-0.377307, 39.467651..."
956,Estació del Nord,Falla Islas Canarias-Trafalgar-Samuel Ros,4293.8,"[[-0.377226, 39.467629], [-0.377307, 39.467651..."
957,Estació del Nord,Falla Ribera-Convento Santa Clara,359.1,"[[-0.377226, 39.467629], [-0.377307, 39.467651..."
958,Estació del Nord,Falla Jerónima Galés-Litógrafo Pascual y Abad,2794.4,"[[-0.377226, 39.467629], [-0.377307, 39.467651..."


In [10]:
df_routes[df_routes.ori.ne('Estació del Nord')&df_routes.des.ne('Estació del Nord')].to_csv("df_routes.csv", index=False)

In [7]:
# Save to CSV
df_routes.to_csv("df_routes.csv", index=False)